# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [12]:
spark

### Import and create SparkSession

In [13]:
from pyspark.sql import SparkSession
spark2 = SparkSession.builder.getOrCreate()
spark2

### Load the PatientInfo.csv file and show the first 5 rows

In [14]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [15]:
df_PatientInfo = spark.read.csv('PatientInfo.csv', inferSchema=True, header = True)

In [16]:
df_PatientInfo.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|

### Display the schema of the dataset

In [17]:
df_PatientInfo.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [18]:
df_PatientInfo_summary = df_PatientInfo.summary()
df_PatientInfo_summary.show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|    5165|
|   mean|2.8636345618679576E9|  NULL|NULL|      NULL|    NULL|          NULL|                NULL|2.2845944015643125E9|1.6772572523506988E7|              NULL|    NULL|
| stddev| 2.074210725277473E9|  NULL|NULL|      NULL|    NULL|          NULL|                NULL|1.5265072953383324E9| 3.093097580985502E8|              N

In [19]:
df_PatientInfo.cache()

23/10/05 16:08:08 WARN CacheManager: Asked to cache already cached data.


DataFrame[patient_id: bigint, sex: string, age: string, country: string, province: string, city: string, infection_case: string, infected_by: string, contact_number: string, symptom_onset_date: string, confirmed_date: date, released_date: date, deceased_date: date, state: string]

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [20]:
df_PatientInfo

DataFrame[patient_id: bigint, sex: string, age: string, country: string, province: string, city: string, infection_case: string, infected_by: string, contact_number: string, symptom_onset_date: string, confirmed_date: date, released_date: date, deceased_date: date, state: string]

In [23]:
df_SurvivedData=df_PatientInfo.select('patient_id').where(col('state')=="released")
df_NotSurvived = df_PatientInfo.select('patient_id').where((col('state')=="isolated") | (col('state')=='deceased'))
print('survived =',df_SurvivedData.count(),'NotSurvived =',df_NotSurvived.count())


survived = 2929 NotSurvived = 2236


In [24]:
df_SurvivedData.show()

+----------+
|patient_id|
+----------+
|1000000001|
|1000000002|
|1000000003|
|1000000004|
|1000000005|
|1000000006|
|1000000007|
|1000000008|
|1000000009|
|1000000010|
|1000000011|
|1000000012|
|1000000014|
|1000000015|
|1000000016|
|1000000017|
|1000000018|
|1000000019|
|1000000020|
|1000000021|
+----------+
only showing top 20 rows



In [25]:
df_released_state = df_PatientInfo.groupBy('state').count()
df_released_state.show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



In [49]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

### Display the number of null values in each column

In [27]:
nulls = {}
for col in df_PatientInfo.columns:
    nullCount = df_PatientInfo.filter(F.col(col).isNull()).count()
    nulls[col] = nullCount
                  

In [28]:
nulls

{'patient_id': 0,
 'sex': 1122,
 'age': 1380,
 'country': 0,
 'province': 0,
 'city': 94,
 'infection_case': 919,
 'infected_by': 3819,
 'contact_number': 4374,
 'symptom_onset_date': 4475,
 'confirmed_date': 3,
 'released_date': 3578,
 'deceased_date': 5099,
 'state': 0}

## Data preprocessing

### Fill the nulls in the deceased_date with the released_date. 
- You can use <b>coalesce</b> function

In [29]:
df_PatientInfo.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



In [30]:
df_PatientInfo_fill = df_PatientInfo.withColumn('deceased_date',F.coalesce('deceased_date','released_date'))
df_PatientInfo.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [31]:
df_PatientInfo_add = df_PatientInfo_fill.withColumn('no_days', date_diff('deceased_date','confirmed_date')) 
df_PatientInfo_add.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)



### Add a is_male column if male then it should yield true, else then False

In [32]:
df_PatientInfo_drop = df_PatientInfo_add.dropna(subset=['sex'])

In [33]:
df_PatientInfo_drop.filter(F.col('sex').isNull()).count()

0

In [34]:
df_PatientInfo_male = df_PatientInfo_drop.withColumn('is_male', (df_PatientInfo['sex']=='male'))
df_PatientInfo_male.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: boolean (nullable = true)



In [35]:
df_PatientInfo_male.filter(F.col('sex').isNull()).count()

0

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task. 
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [36]:
def add_col(str_):
    if str_ != 'released':
        return True
    else:
        return False
    

In [39]:
convertUDF = udf(add_col,BooleanType())

In [40]:
df_PatientInfo.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



In [41]:
df_PatientInfo_UDF = df_PatientInfo_male.withColumn('is_dead', convertUDF(df_PatientInfo['state']))

In [42]:
df_PatientInfo_UDF.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



In [43]:
df_PatientInfo_UDF.filter(F.col('sex').isNull()).count()

0

In [55]:
df_PatientInfo_UDF.filter(F.col('age').isNull()).count()

261

In [ ]:
add_col('released')

In [44]:
df_PatientInfo_dead = df_PatientInfo_UDF.withColumn('is_dead', df_PatientInfo['state']!='released')
df_PatientInfo_dead.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



In [45]:
df_PatientInfo_dead.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|   true|  false|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|   true|  false|
|1000000003|  male|5

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [46]:
df_PatientInfo_ages = df_PatientInfo_dead.withColumn('age',split(df_PatientInfo_dead['age'],'s')[0])

In [ ]:
df_PatientInfo_ages.show()

### Change age, and no_days  to be typecasted as Double

In [47]:
from pyspark.sql.types import *


In [50]:
df_PatientInfo_Double = df_PatientInfo_ages.withColumn('age',col('age').cast(DoubleType())).withColumn('no_days',col('no_days').cast(DoubleType()))
df_PatientInfo_Double.show()

+----------+------+----+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex| age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+----+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50.0|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|   13.0|   true|  false|
|1000000002|  male|30.0|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|   32.0|   true|  false|
|100000000

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [51]:
df_PatientInfo_Drop = df_PatientInfo_Double.drop("patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case")


In [52]:
df_PatientInfo_Drop.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_male: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



In [ ]:
df_PatientInfo_Drop.filter(col('is_dead'))

### Recount the number of nulls now

In [53]:
nulls_final = {}
for col in df_PatientInfo_Drop.columns:
    nullCount = df_PatientInfo_Drop.filter(F.col(col).isNull()).count()
    nulls_final[col] = nullCount

In [54]:
nulls_final

{'age': 261, 'province': 0, 'is_male': 0, 'is_dead': 0}

In [ ]:
df_PatientInfo_Drop.groupBy('is_male').count().show()


## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [ ]:
df_PatientInfo.createOrReplaceTempView("Myview")

### Use SELECT statement to select all columns from the dataframe and show the output.

In [ ]:
spark.sql("""select * from Myview""").show()

### *Using SQL commands*, limit the output to only 5 rows 

In [ ]:
spark.sql("""select * from Myview limit 5""").show()

### Select the count of males and females in the dataset

In [ ]:
spark.sql("""select sex, count(patient_id) from Myview group by(sex)""").show()

### How many people did survive, and how many didn't?

In [ ]:
spark.sql("""select
    (select count(patient_id) from Myview where state = 'released') as survived,
    count(patient_id) as Dead from Myview where state = 'deceased' or state = 'isolated';
""").show()

In [ ]:
spark.sql("""select count(patient_id) from Myview where state == 'released' union
             select count(patient_id) from Myview where state == 'deceased' or state == 'isolated'""").show()

### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [ ]:
spark.sql("""select cast(SUBSTRING(age, 0,2) as double )as age from Myview """).show()

In [ ]:
spark.sql("""select cast(SUBSTRING(age, 0,2) as double )as age,province,sex,state from Myview """).show()

In [ ]:
df_sql_final = spark.sql("""select cast(SUBSTRING(age, 0,2) as double )as age,province,sex,state from Myview """)
df_sql_final.show()          

23/10/05 16:03:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
